# RooFit Tutorial: Introduction to Unbinned Likelihood Models

Jonas Rembser (CERN), 2023

## Setup

Import ROOT and NumPy:

In [ ]:
import ROOT
import numpy as np

Silence the RooFit logging:

In [ ]:
ROOT.RooMsgService.instance().setGlobalKillBelow(ROOT.RooFit.FATAL)

## The basics

Mathematical concepts are represented by C++ objects:


![roofit_classes.png](roofit_classes.png)

### Creating your first RooFit model

In [ ]:
import ROOT

Observable:

In [ ]:
x = ROOT.RooRealVar("x", "x", 0, 0, 10)

Parameters:

In [ ]:
mean = ROOT.RooRealVar("mean", "mean of gaussian", 5, 0, 10)
sigma = ROOT.RooRealVar("sigma", "width of gaussian", 1, 0.1, 10)

Gaussian PDF:

In [ ]:
gauss = ROOT.RooGaussian("gauss", "gaussian PDF", x, mean, sigma)

PDF inspection:

In [ ]:
gauss.Print("t")

### Toy dataset generation and fitting

Generate a toy dataset with 9000 entries sampled from the Gaussian PDF:

In [ ]:
data = gauss.generate({x}, 9000)

In [ ]:
data.Print()

Fit the PDF to the toy data, saving the fit result:

In [ ]:
fit_result = gauss.fitTo(data, PrintLevel=-1, Save=True)

In [ ]:
fit_result.Print()

Inspect the correlation of your model parameters:

In [ ]:
fit_result.correlationMatrix().Print()

## Plotting the data and the model

Create a `RooPlot` object on which the data and PDF is plotted:

In [ ]:
x_frame = x.frame(Title="Gaussian PDF with data")

In [ ]:
data.plotOn(x_frame)
gauss.plotOn(x_frame);

Draw the RooPlot on a `TCanvas`:

In [ ]:
c1 = ROOT.TCanvas("c1", "c1", 500, 300)
x_frame.Draw()
c1.Draw()

### Importing RooFit datasets from a ROOT file

Export the dataset to a ROOT file so we can show how to import it:

In [ ]:
data.convertToTreeStore()

output_file = ROOT.TFile("dataset.root", "RECREATE")
data.store().tree().SetName("mytree")
data.store().tree().SetTitle("My measured data")
data.store().tree().Write()
output_file.Close()

data.convertToVectorStore()

ROOT file with a TTree that stores the data you want to fit:

In [ ]:
input_file = ROOT.TFile("dataset.root", "READ") # file contains a TTree called "mytree"

Import to `RooDataSet` using the constructor that takes a TTree:

In [ ]:
dataset = ROOT.RooDataSet("dataset", "dataset", input_file.mytree, {x})

Don't forget to close the file:

In [ ]:
input_file.Close()

You will see again the dataset with the observable `x`:

In [ ]:
dataset.Print()

### Exporting your RooFit datasets

You can export a RooDataSet to NumPy or Pandas:

In [ ]:
df = data.to_pandas()

In [ ]:
df

## Composite PDFs

Composite PDF: model with mutiple components, like signal and background.

Adding random "background" values sampled from exponential PDF to the dataset. We use NumPy for that to showcase how NumPy arrays can be imported to `RooDataSet`:

In [ ]:
exp_tau = -0.18
arr_new = np.concatenate([data.to_numpy()["x"],
                          np.random.exponential(-1./exp_tau, size=4* data.numEntries())])

Import the new array back to a RooDataSet:

In [ ]:
data_x = ROOT.RooDataSet.from_numpy({"x" : arr_new[arr_new < x.getMax()]}, {x}, name="data_new")

In [ ]:
data_x.Print()

Visualize the dataset:

In [ ]:
x_frame = x.frame(Title="Plotting Gaussian plus exp. background")
data_x.plotOn(x_frame)

c2 = ROOT.TCanvas()
x_frame.Draw()
c2.Draw()

### Creating the composite fit model

Create exponential PDF with parameter "tau":

In [ ]:
tau = ROOT.RooRealVar("tau", "tau", -0.2, -10.0, -0.01)
expo = ROOT.RooExponential("expo", "expo", x, tau)

Define parameters for the number of signal and background events:

In [ ]:
n_sig = ROOT.RooRealVar("n_sig", "n_sig", 10000, 1000, 100000)
n_bkg = ROOT.RooRealVar("n_bkg", "n_bkg", 50000, 5000, 500000)

Composite model that automatically includes a Poisson term for the total number of events:

In [ ]:
model = ROOT.RooAddPdf("model", "model", [gauss, expo], [n_sig, n_bkg])

Do the fit:

In [ ]:
fit_result = model.fitTo(data_x, PrintLevel=-1, Save=True)
fit_result.Print()

## Creating a nice plot

Create RooPlot and draw data, PDF, and components:

In [ ]:
x_frame = x.frame(Title="Gaussian plus exp. background")

data_x.plotOn(x_frame, Name="data")

model.plotOn(x_frame, Components=gauss, LineColor="r", LineStyle="--", Name="gauss")
model.plotOn(x_frame, Components=expo, LineColor="k", LineStyle="--", Name="expo")
model.plotOn(x_frame, Name="model");

Add a legend:

In [ ]:
legend = ROOT.TLegend(0.7, 0.55, 0.92, 0.87)
legend.SetBorderSize(0)
legend.SetFillStyle(0)
legend.AddEntry(x_frame.findObject("data"), "data", "P")

for name in ["model", "gauss", "expo"]:
    legend.AddEntry(x_frame.findObject(name), name, "L")

Create a second frame with the residuals:

In [ ]:
resid_hist = x_frame.residHist()

resid_frame = x.frame(Title=";x;residuals")
resid_frame.addPlotable(resid_hist, "P")

Create a canvas that is divided into two drawing pads:

In [ ]:
c3 = ROOT.TCanvas("c3", "c3", 600, 600)
c3.Divide(1, 2)

First pad is for the main plot and the legend:

In [ ]:
pad_1 = c3.cd(1)
x_frame.Draw()
legend.Draw()
pad_1.SetPad(0.0, 0.2, 1, 1)

Second pad is for the residuals:

In [ ]:
pad_2 = c3.cd(2)
pad_2.SetPad(0., 0.0, 1, 0.25)
resid_frame.Draw()
resid_frame.GetXaxis().SetLabelSize(0.12)
resid_frame.GetYaxis().SetLabelSize(0.12)
resid_frame.GetYaxis().SetTitleSize(0.12)
resid_frame.GetYaxis().SetTitleOffset(0.25)

Draw the canvas:

In [ ]:
c3.Draw()

## Template fits with convolutions

A **template** PDF is based on *histogram shape*, and not expressed by an analytical function.

Imagine you have a histogram giving the expected signal shape:

In [ ]:
template_hist = ROOT.TH1D("h1", "h1", 100, 0, 10)
f1 = ROOT.TF1("f1", "std::exp(-std::abs((x-5)))", 0, 10)
template_hist.FillRandom("f1", 100000)

In [ ]:
c4 = ROOT.TCanvas("c4", "c4", 600, 400)
template_hist.Draw()
c4.Draw()

Getting the template PDF into RooFit:

1. Create a corresponding observable:

In [ ]:
y = ROOT.RooRealVar("y", "y", 0, 0, 10)

2. Convert the `TH1` into a `RooDataHist`:

In [ ]:
roo_template_hist = ROOT.RooDataHist("roo_template_hist", "roo_template_hist", y, template_hist)

3. Create a `RooHistPdf` based of the RooFit histogram:

In [ ]:
sig_raw_y = ROOT.RooHistPdf("sig__raw_y", "sig_raw_y", y, roo_template_hist)

### Creating a full composite model

Construct a `RooGaussian` to model detector resolution effects:

In [ ]:
resolution = ROOT.RooRealVar("resolution", "resolution", 0.2, 0.1, 1.0)
sig_smearing_y = ROOT.RooGaussian("sig_smearing_y", "sig_smearing_y", y, ROOT.RooFit.RooConst(0.0), resolution)

The signal PDF is a convolution of the template and the resolution function:

In [ ]:
sig_y = ROOT.RooFFTConvPdf("sig_y", "sig_y", y, sig_raw_y, sig_smearing_y)

For the background, we use a Chebychev polynomial:

In [ ]:
bkg_y = ROOT.RooChebychev("bkg_y", "bkg_y", y, [-0.5, 0.1])

Finally, create **RooAddPdf** for the composite model:

In [ ]:
model_y = ROOT.RooAddPdf("model_y", "model_x", [sig_y, bkg_y], [n_sig, n_bkg])

Creating toy dataset and fitting

In [ ]:
data_y = model_y.generate(y)

In [ ]:
fit_result = model_y.fitTo(data_y, PrintLevel=-1, Save=True)
fit_result.Print()

Plotting the model and the toy dataset

In [ ]:
y_frame = y.frame(Title="Model for y")

data_y.plotOn(y_frame)
model_y.plotOn(y_frame)

c5 = ROOT.TCanvas()
y_frame.Draw()
c5.Draw()

### Overview of other PDF types

RooFit provides a collection of standard PDF classes, e.g.:

![roofit_pdfs.png](roofit_pdfs.png)

Easy to **extend the library**: each pdf is a separate C++ class

## Multivariate fit

We have now modeled two observables:
* `x` with Gaussian signal and exponential background
* `y` with smeared template signal and Chebychev background

Create 2D model $p(x,y) = p(x) p(y)$ for signal and background

In [ ]:
model_sig_xy = ROOT.RooProdPdf("model_sig_xy", "model_sig_xy", [gauss, sig_y])
model_bkg_xy = ROOT.RooProdPdf("model_bkg_xy", "model_bkg_xy", [expo, bkg_y])

Yet again, a RooAddPdf for the final model

In [ ]:
model_xy = ROOT.RooAddPdf("model_xy", "model_xy", [model_sig_xy, model_bkg_xy], [n_sig, n_bkg])

Generating a 2D toy dataset

In [ ]:
data_xy = model_xy.generate({x, y}, 10000)

Visualize the 2D data in a LEGO plot

In [ ]:
histo_xy = data_xy.createHistogram("histo_xy", x, Binning=25, YVar=dict(var=data_xy.get()["y"], Binning=15))
histo_xy.SetTitle("")

c6 = ROOT.TCanvas()
histo_xy.Draw("LEGO2")
c6.Draw()

Fitting the 2D model

By now you know how it works:

In [ ]:
fit_result_xy = model_xy.fitTo(data_xy, PrintLevel=-1, Save=True)

Fit result has all parameters we saw before:

In [ ]:
fit_result_xy.Print()

### Final visualization of 2D model and data

In [ ]:
x_frame = x.frame(Title="Model for x")
y_frame = y.frame(Title="Model for y")

data_xy.plotOn(x_frame)
model_xy.plotOn(x_frame)

data_xy.plotOn(y_frame)
model_xy.plotOn(y_frame)

c7 = ROOT.TCanvas("c7", "c7", 800, 400)
c7.Divide(2)
c7.cd(1)
x_frame.Draw()
c7.cd(2)
y_frame.Draw()
c7.Draw()

### Likelihood scans

It is very useful to plot and inspect the NLL and also the profiled NLL. For this, you can use `createNLL` to get a RooFit object that represents a likelihood directly. More examples can be found in the [rf605_profilell tutorial](https://root.cern/doc/master/rf605__profilell_8py.html).

In [ ]:
# Create likelihood function
nll = model_xy.createNLL(data_xy, BatchMode="cpu")
# The "BatchMode" is a performance optimization, it can also be used in `fitTo`

# Minimize likelihood such that all other parameters (nuisance parameters) are at the best fit value.
minimizer = ROOT.RooMinimizer(nll)
minimizer.setPrintLevel(-1)
minimizer.minimize("Minuit", "")

# Make RooPlot for our parameter of interest, let's say n_sig
window = 5 * n_sig.getError()
n_sig_frame = n_sig.frame(Bins=10, Range=(n_sig.getVal() - window, n_sig.getVal() + window))

# Plot likelihood scan in parameter n_sig
nll.plotOn(n_sig_frame, ShiftToZero=True)

# Plot the profile likelihood in n_sig.
# Now, the nuisance parameter are optimized for each scanned value of n_sig.
pll_n_sig = nll.createProfile([n_sig])
pll_n_sig.plotOn(n_sig_frame, LineColor="kRed")

# Set y axis limits
n_sig_frame.SetMinimum(0)
n_sig_frame.SetMaximum(5)

c8 = ROOT.TCanvas()
n_sig_frame.Draw()
c8.Draw()

## Model inspection

You already know the `Print("t")` function:

In [ ]:
model.Print("t")

### Graphical model visualization

Export the model to a `GraphViz` file

In [ ]:
model_xy.graphVizTree("model.dot")

Turn the `GraphViz` file into a GIF file in the shell:

In [ ]:
!dot -Tpng -o model.png model.png

![model.png](model.png)

## The RooWorkspace

The RooFit objects can be managed by a `RooWorkspace`:

In [ ]:
ws = ROOT.RooWorkspace("myworkspace")

You can for example import an existing model:

In [ ]:
ws.Import(model_xy);

You can `Print` the workspace for inspecting its content:

In [ ]:
ws.Print()

Access any object in the RooWorkspace:

In [ ]:
ws["model_xy"].Print()

Save the workspace to a ROOT file to reuse the model later

In [ ]:
ws.writeToFile("myworkspace.root");

## Exercises

1. Further improve the plot with the pull distribution by visualizing also the post-fit uncertainty of the model. Figure out how to do this by reading the documentation of [RooAbsPdf::plotOn()](https://root.cern.ch/doc/master/classRooAbsPdf.html#aa0f2f98d89525302a06a1b7f1b0c2aa6).

2. Look at the [rf203_ranges.py RooFit tutorial](https://root.cern/doc/master/rf203__ranges_8py.html) to learn how to restrict the fit to a subrange. Redo the convoluted template fit to the $y$ variable, but restricted to the range from 3 to 7.

   Why does the uncertainty of the `resolution` parameter increase, even though we are not excluding that much signal and `resolution` doesn't affect the background?

3. Interpret the likelihood plot over `n_sig`. Why is the profile NLL always below the other plotted NLL?

4. In a fresh notebook, open the `RooWorkspace` we wrote to disk and create new toy data according to the 2D model. Re-fit the model to the new toy dataset.

5. Which parameters are strongly (anti)correlated in the final 2D fit? Can you explain why?